# **Title: Loan Default Risk Analysis**

### **Author: Sandeep kumar**
##### **Contribution**    - Individual


## 🎯 **Problem Statement**
### **1. What is the business problem?**

The lending institution needs to understand the risk associated with issuing loans. A significant portion of borrowers fail to repay their loans, resulting in financial losses. The goal is to identify the patterns and factors that influence loan repayment behavior and highlight areas where risk is higher.


### **2. What are we trying to understand?**

We aim to analyze historical loan data to:

* Measure the overall loan default rate

* Identify borrower characteristics and loan attributes linked to higher risk

* Compare repayment behavior across different income levels, grades, terms, and purposes

* Understand trends over time (year/month)

* Reveal which segments of borrowers are most likely to default

* This analysis helps uncover why certain loans default and what factors significantly contribute to repayment failure.


### **3. Who benefits from this analysis?**

This analysis benefits multiple stakeholders:

* Risk Management Teams – to better assess borrower risk and reduce expected losses

* Credit Underwriting Teams – to refine approval criteria and improve decision-making

* Finance & Strategy Teams – to understand portfolio performance and design better policies

* Lenders/Organizations – to improve profitability by reducing high-risk lending

* Borrowers – through more accurate and fair loan terms based on their risk profile



### **4. Core Portfolio KPIs**

* Total Loan Applications

* Total Funded Amount

* Total Amount Received

* Average Interest Rate

* Average DTI

### **5. Good vs Bad Loan Analysis**

* Default vs good loan distribution

* Risk factors influencing default (DTI, interest rate, term, grade, purpose)

## **GitHub Link -**

https://github.com/SandyCherry96/Loan-Default-Risk-Analysis.git


## **1.Dataset Schema Mapping**


#### A. Loan Information:
| Field         | Meaning                                   |
| ------------- | ----------------------------------------- |
| id            | Unique loan identifier                    |
| loan_amount   | Total loan amount approved / funded       |
| installment   | Monthly EMI the borrower must pay         |
| term          | Loan duration (typically 36 or 60 months) |
| int_rate      | Interest rate charged on the loan         |
| total_payment | Total amount the borrower has paid so far |


#### B. Loan Status & Repayment Tracking
| Field                 | Meaning                                                            |
| --------------------- | ------------------------------------------------------------------ |
| loan_status           | Current state of the loan (Fully Paid, Charged Off, Current, etc.) |
| last_payment_date     | Date of last payment received                                      |
| next_payment_date     | Upcoming scheduled payment date                                    |
| last_credit_pull_date | Date lender last reviewed the borrower’s credit report             |


#### C. Borrower Demographics & Employment
| Field            | Meaning                                           |
| ---------------- | ------------------------------------------------- |
| emp_title        | Borrower's job title                              |
| emp_length       | Years of employment (e.g., 1 year, 10+ years)     |
| home_ownership   | Home ownership type (Rent, Own, Mortgage)         |
| address_state    | State where the borrower lives                    |
| annual_income    | Borrower's reported yearly income                 |
| application_type | Whether the application is individual or joint    |
| member_id        | Internal borrower identifier (unique to platform) |

#### D. Borrower Financial Health & Risk Factors
| Field               | Meaning                                                                          |
| ------------------- | -------------------------------------------------------------------------------- |
| dti                 | Debt-to-Income ratio → how much debt borrower has relative to income             |
| total_acc           | Total number of credit accounts (credit cards, loans, etc.)                      |
| grade               | Loan grade assigned by lender (A–G) based on borrower creditworthiness           |
| sub_grade           | More detailed grading within grade (A1–G5)                                       |
| verification_status | Whether borrower’s income was verified (Verified, Not Verified, Source Verified) |

#### E. Loan Purpose
| Field   | Meaning                                                                          |
| ------- | -------------------------------------------------------------------------------- |
| purpose | Reason for taking the loan (credit card, small business, home improvement, etc.) |




## **2.Import required libraries**


In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 

#### **Dataset Loading**

In [ ]:
path = "C:\\Users\\sanrkin\\Downloads\\financial_loan.xlsx - financial_loan.csv"
df = pd.read_csv(path)

#### **Dataset First View**

In [ ]:
df['annual_income']

In [ ]:
df.head()

In [ ]:
df.tail()

#### **Understanding Your Variables**

#### **Dataset Columns**

In [ ]:
print(df.columns)

#### **Dataset Rows & Columns count**

In [ ]:
print("Number of Rows:",df.shape[0])
print("Number of Columns:",df.shape[1])

#### **Dataset Information**

In [ ]:
df.info()

In [ ]:
df.describe()

### **🔍 Data Quality Check**

#### **Duplicate Values**

In [ ]:
print("Total duplicate rows:", df.duplicated().sum())

#### **Missing Values/Null Values**

In [ ]:
df.isnull().sum()

##### **📌 Key Insights:**

Only emp_title has missing values (1,438 null entries)

In [ ]:
(df.isnull().sum()/len(df))*100

In [ ]:
df['emp_title'].fillna('Unknown',inplace=True)
df.isnull().sum()

##### **📌 Key Insights:**
I filled missing emp_title with ‘Unknown’ to keep all data, avoid empty categories, and treat missing job titles as a meaningful group

### **What did you know about your dataset?**

* The dataset contains 38,576 rows and 24 columns, each representing an individual loan record.


* It includes borrower details, loan characteristics, and repayment information.


* The dataset is well-structured and suitable for analyzing loan default patterns and risk factors.

* Only emp_title has missing values


#### **Convert Date Columns**

In [ ]:
date_columns = ['issue_date', 'last_credit_pull_date', 
                'last_payment_date', 'next_payment_date']
for col in date_columns:
    df[col] = pd.to_datetime(df[col])

#### **📌 Key Insights:**
I converted all date columns into proper datetime format so that I can sort, filter, and analyze loans based on time, and calculate important time-based metrics like trends

In [ ]:
df['home_ownership'] = df['home_ownership'].str.strip().str.title()
df['purpose'] = df['purpose'].str.replace("-", " ").str.title()

In [ ]:
# ----------------------------------
# 1. Loans where total payment >= loan amount
# ----------------------------------

if 'loan_amount' in df.columns and 'total_payment' in df.columns:
    condition = df['total_payment'] >= df['loan_amount']
    
    count = condition.sum()
    pct = round((count / len(df)) * 100, 2)
    
    print("Loans with total_payment ≥ loan_amount:", count, "rows")
    print("Percentage of such loans:", pct, "%")


# ----------------------------------
# 2. DTI check (DTI should not be negative)
# ----------------------------------

if 'dti' in df.columns:
    neg_dti_count = (df['dti'] < 0).sum()
    print("\nNegative DTI rows:", neg_dti_count)


# ----------------------------------
# 3. Interest rate check (should not be negative)
# ----------------------------------

if 'int_rate' in df.columns:
    neg_int_count = (df['int_rate'] < 0).sum()
    print("\nNegative interest rate rows:", neg_int_count)


### **Outlier Detection & Treatment**

In [ ]:
## Outlier Analysis
num_cols = df.select_dtypes(include=['int64','float64']).columns

plt.figure(figsize=(16, 6))
sns.boxplot(data=df[num_cols], orient='v', showfliers=True)

plt.title("Outlier Detection Across Numerical Features", fontsize=16, weight='bold')
plt.xticks(fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.4)

plt.tight_layout()
plt.show()

### **Insights from the Boxplot: Outlier Detection Across Numerical Features**

**1. Why did i pick this specific chart?**

* A boxplot is the most effective way to visually detect outliers across multiple numerical variables at once.

* It shows the distribution, median, IQR, and extreme values clearly.


**2. What insights did we find from the chart?**

**A. Heavy Outliers in Annual Income.**

These values are far above normal income levels, suggesting:

* Possible data-entry errors

* Rare high-income customers

* Highly skewed distribution

**B. ID & Member_ID are Not Useful for Modeling**

* id and member_id also appear as "outliers" but these are identifier variables, not meaningful numerical features.They should be dropped.

**C. Loan-Related Features Are Well-Behaved**

Features such as:

* loan_amount

* installment

* int_rate

* dti

* total_acc

* total_payment

**3. Will the gained insights help create positive business impact?**

Absolutely yes.

These insights drive major business improvements:

**Better Risk Modeling**

* Treating outliers prevents biased loan-default predictions.

* Cleaner data → more accurate ML models → fewer bad loans.

**Improved Fraud Detection**

Unrealistically high income values may point to:

* fraudulent applications

* misreported income


Outliers in financial datasets represent real-world customer behavior such as very high incomes, very large loan amounts, or extreme DTI ratios. These are not errors — they are exactly the cases that determine credit risk. Removing outliers would remove high-risk borrowers, and reduce real-world accuracy. Therefore, we keep outliers and handle them using transformations rather than deleting them.


## **3.Business KPI's**


### **1.Total Loan Applications**

In [ ]:
Total_loan_applications = df['id'].count()
print(f"Total Loan Applications:",Total_loan_applications)

### **2.Total Funded Amount**

In [ ]:
million = 1000000
Total_funded_amount = df['loan_amount'].sum()
print(f"\n💰Total Funded Amount: {Total_funded_amount / million:.2f} Million $")

### **3.MTD Loan Applications**

In [ ]:
latest_issue_date = df['issue_date'].max()
latest_year  = latest_issue_date.year
latest_month  = latest_issue_date.month

mtd_data = df[
    (df['issue_date'].dt.year == latest_year) & 
    (df['issue_date'].dt.month == latest_month)]

mtd_loan_applications = mtd_data['id'].count()
print("\n📆 Month-To-Date (MTD) Loan Applications")
print(f"Latest Month: {latest_issue_date.strftime('%B %Y')}")
print(f"Total MTD Loan Applications: {mtd_loan_applications:,}")

### **4.MTD Funded Amount**

In [ ]:
mtd_funded_amount = mtd_data['loan_amount'].sum()
print(f"\n💰MTD Funded Amount:{mtd_funded_amount / million:.2f} Million" )

### **5.Total Amount Received**

In [ ]:
Total_amount_received = df['total_payment'].sum()
print(f"\n💰Total received Amount: {Total_amount_received / million:.2f} Million $")

### **6.MTD Amount Received**

In [ ]:
mtd_amount_received = mtd_data['total_payment'].sum()
print(f"\n💰MTD received Amount: {mtd_amount_received / million:.2f} Million $")

### **7.Average Interest Rate**

In [ ]:
Average_interest_rate = df['int_rate'].mean()
print(f"\nAverage Interest Rate: {Average_interest_rate:.2f}%")

### **8.Average DTI**

In [ ]:
Average_DTI = df['dti'].mean()
print(f"\nAverage Debt to income: {Average_DTI:.2f}%")

## **4.Good Loan KPIs**

In [ ]:

Good_loans = df[df['loan_status'].isin(['Fully Paid','Current'])]

Good_loan_applications = Good_loans['id'].count()
print(f"\nTotal Good Loan Applications:",Good_loan_applications)

Good_load_funded_amount = Good_loans['loan_amount'].sum()
print(f"\n💰Good loans Funded Amount:{Good_load_funded_amount/million:.2f} Million$")

Good_load_received_amount = Good_loans['total_payment'].sum()
print(f"\n💰Good loans received Amount:{Good_load_received_amount/million:.2f} Million$")

Good_loans_percentage = (Good_loan_applications/Total_loan_applications)*100
print(f"\nPerecentage of Good Loan Applications:{Good_loans_percentage:.2f}%")

## **Bad Loan KPIs**

In [ ]:
Bad_loans = df[df['loan_status'].isin(['Charged Off'])]

Bad_loan_applications = Bad_loans['id'].count()
print(f"\nTotal Bad Loan Applications:",Bad_loan_applications)

Bad_load_funded_amount = Bad_loans['loan_amount'].sum()
print(f"\n💰Bad loans Funded Amount:{Bad_load_funded_amount/million:.2f} Million$")

Bad_load_received_amount = Bad_loans['total_payment'].sum()
print(f"\n💰Bad loans received Amount:{Bad_load_received_amount/million:.2f} Million$")

Bad_loans_percentage = (Bad_loan_applications/Total_loan_applications)*100
print(f"\nPerecentage of Bad Loan Applications:{Bad_loans_percentage:.2f}%")


In [ ]:
# 1️⃣ Create 'loan_category' if it doesn't exist
# Good Loan = Fully Paid / Current
# Bad Loan = Charged Off (you can adjust based on your project)

if 'loan_category' not in df.columns:
    good_status = ['Fully Paid', 'Current']
    bad_status = ['Charged Off']

    df['loan_category'] = np.where(
        df['loan_status'].isin(good_status), 'Good Loan',
        np.where(df['loan_status'].isin(bad_status), 'Bad Loan', 'Other')
    )

# 2️⃣ Create summary table
summary_stats = pd.DataFrame({
    'Metric': [
        'Average Loan Amount',
        'Average Interest Rate',
        'Average DTI',
        'Count',
        'Percentage'
    ]
})

# Only use Good Loan / Bad Loan categories
for category in ['Good Loan', 'Bad Loan']:
    category_data = df[df['loan_category'] == category]
    
    if len(category_data) == 0:
        # If category doesn't exist, fill with NA / 0
        summary_stats[category] = ["N/A", "N/A", "N/A", "0", "0.0%"]
        continue

    percentage = (len(category_data) / len(df)) * 100

    summary_stats[category] = [
        f"${category_data['loan_amount'].mean():,.2f}",
        f"{category_data['int_rate'].mean():.2f}%",
        f"{category_data['dti'].mean():.2f}",
        f"{len(category_data):,}",
        f"{percentage:.1f}%"
    ]

print("\nSummary Statistics by Loan Category:")
print(summary_stats.to_string(index=False))


## **Analysis by Loan Status (Good vs Bad Loans)**

In [ ]:
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# 5.1 loan category
df['loan_category'] = df['loan_status'].map(
    lambda x: 'Good Loan' if x == 'Fully Paid' else 'Bad Loan'
)
colors = ['#2ecc71', '#e74c3c']  

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

loan_distribution = df['loan_category'].value_counts()

# Pie chart
ax1.pie(
    loan_distribution, 
    labels=loan_distribution.index,
    autopct='%1.1f%%',
    colors=colors,
    explode=(0, 0.1),
    textprops={'fontweight': 'bold'}    
)
ax1.set_title('Distribution of Loan Categories', fontweight='bold')

# Bar plot
sns.barplot(
    x=loan_distribution.index, 
    y=loan_distribution.values,
    palette=colors,
    ax=ax2
)
ax2.set_title('Count of Loans by Category', fontweight='bold')
ax2.set_xlabel('Loan Category', fontweight='bold')
ax2.set_ylabel('Count', fontweight='bold')

for i, v in enumerate(loan_distribution.values):
    ax2.text(i, v, f'{v:,}', ha='center', va='bottom', fontweight='bold')

for i, v in enumerate(loan_distribution.values):
    ax2.text(i, v, f'{v:,}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

#### **📌 Key Insights:**
##### **Why did i pick this chart?**

The pie and bar charts clearly show the proportion and count of Good Loans vs Bad Loans. These visuals are simple, easy to interpret, and help quickly understand the overall loan portfolio quality.

##### **What are the insights from the chart?**

* 83.3% are Good Loans, showing most borrowers repay successfully.

* 16.7% are Bad Loans, indicating a significant default segment that needs attention.

* The large gap between the two categories shows overall portfolio stability but highlights risk areas.

##### **Will these insights help create a positive business impact?**

Yes. These insights help lenders improve credit policies, adjust risk thresholds, and focus on reducing defaults, which can directly increase profitability.

##### **Are there any insights indicating negative growth? Why?**

Yes. The 16.7% Bad Loan rate is a negative indicator because it represents financial losses and higher risk exposure. Reducing this segment is crucial for long-term growth.

## **Interest Rate Analysis**

In [ ]:
plt.figure(figsize=(8,4))

sns.boxplot(
    x='loan_category',
    y='int_rate',
    data=df,
    showfliers=False,   
    linewidth=1.2,
    palette=colors
)

plt.title("Interest Rate by Loan Category", fontsize=14, weight='bold')
plt.xlabel("Loan Category", fontsize=12)
plt.ylabel("Interest Rate (%)", fontsize=12)

plt.grid(True, linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()

#### **📌 Key Insights:**

##### **Why did i pick this chart?**

A boxplot is ideal for comparing the distribution of interest rates between Good Loans and Bad Loans. It clearly shows the median, spread, and outliers, helping us understand how interest rate levels differ between risk groups.

##### **What insights are found from the chart?**

* Bad Loans have higher interest rates, with a higher median and wider spread.

* Good Loans have lower interest rates, showing better credit quality.

* The variation in Bad Loans suggests that risky borrowers are charged higher rates, but many still default despite higher pricing.

##### **Will these insights help create a positive business impact?**

Yes. This insight helps the lender evaluate whether interest rate pricing aligns with actual borrower risk. It can guide better risk-based pricing and improve underwriting strategies.

##### **Are there any insights indicating negative growth? Why?**

Yes. Even at higher interest rates, many borrowers still default. This means that increasing interest rates alone does not fully compensate for the risk and may even worsen affordability, leading to defaults.

This suggests a need for better credit assessment, not just higher pricing

## **DTI Analysis**

In [ ]:
plt.figure(figsize=(8,4))

sns.boxplot(
    x='loan_category',
    y='dti',
    data=df,
    showfliers=False,
    linewidth=1.2,
    palette=colors
)

plt.title("DTI by Loan Category", fontsize=14, weight='bold')
plt.xlabel("Loan Category", fontsize=12)
plt.ylabel("Debt-to-Income Ratio", fontsize=12)

plt.grid(True, linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()

### **📌 Key Insights: DTI by Loan Category**

##### **Why did i pick this chart?**

A boxplot is ideal for comparing the distribution of DTI (Debt-to-Income ratio) between Good Loans and Bad Loans. It highlights the median, spread, and variation in financial stability between the two groups.

##### **What insights are found from the chart?**

* Bad Loan customers have a higher median DTI, meaning they carry more debt relative to their income.

* Higher DTI indicates greater financial stress, making repayment more difficult.

* Good Loan customers show lower and more stable DTI levels, indicating stronger repayment capacity.

##### **Will these insights help create a positive business impact?**

Yes. This insight helps refine risk assessment. Lenders can set DTI thresholds or tighten approvals for high-DTI borrowers, reducing future default rates.

##### **Are there any insights indicating negative growth? Why?**

Yes. The higher DTI levels among Bad Loan customers suggest that many borrowers who default are already financially overextended.
If such borrowers continue to be approved, default rates will keep rising, which negatively impacts profitability.


## **👉 5. Exploratory Data Analysis**


## **1.Univariate Analysis**


### **Loan Amount Distribution**

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Histogram plot
sns.histplot(
    data=df,
    x='loan_amount',
    bins=30,
    kde=True,              
    linewidth=1.2,
    ax=ax1
)
ax1.set_title("Distribution of Loan Amounts", fontsize=14, weight="bold")
ax1.set_xlabel("Loan Amount", fontsize=12)
ax1.set_ylabel("Frequency", fontsize=12)
ax1.grid(True, linestyle="--", alpha=0.3)

# Box plot
sns.boxplot(
    data=df,
    y='loan_amount',      
    linewidth=1.2,
    ax=ax2
)
ax2.set_title("Boxplot of Loan Amounts", fontsize=14, weight="bold")
ax2.set_ylabel("Loan Amount", fontsize=12)
ax2.grid(True, linestyle="--", alpha=0.3)

plt.tight_layout()
plt.show()


### **📌 Key Insights: Loan Amount Distribution**

##### **Why did i pick this chart?**

The histogram shows how loan amounts are distributed across borrowers, while the boxplot helps identify the median, spread, and outliers. Together, they give a clear picture of typical loan sizes and extreme values in the dataset.

##### **What insights are found from the chart?**

* Most loans fall between $5,000 and $15,000, meaning this is the most common borrowing range.

* The distribution is right-skewed, with fewer borrowers taking very large loans.

* The boxplot shows several high-value outliers above $25,000–$30,000.

* The median loan amount is around $10,000, indicating the central borrowing tendency.

##### **Will these insights help create a positive business impact?**

Yes. Loan amount patterns help lenders understand borrowing behavior, optimize loan product offerings, and design risk-based pricing or credit policies for different loan-size segments.

##### **Are there any insights indicating negative growth? Why?**

Yes. The presence of many high-value outliers could signal potential risk because larger loan amounts generally carry a higher probability of default and higher loss impact. These need stricter underwriting or additional verification.

## **Interest Rate Distribution**

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Histogram plot
sns.histplot(
    data=df,
    x='int_rate',
    bins=30,
    kde=True,              
    linewidth=1.2,
    ax=ax1
)
ax1.set_title("Distribution of Interest Rates", fontsize=14, weight="bold")
ax1.set_xlabel("Interest Rate (%)", fontsize=12)
ax1.set_ylabel("Frequency", fontsize=12)
ax1.grid(True, linestyle="--", alpha=0.3)

# Box plot
sns.boxplot(
    data=df,
    y='int_rate',    
    linewidth=1.2,
    ax=ax2
)
ax2.set_title("Boxplot of Interest Rates", fontsize=14, weight="bold")
ax2.set_ylabel("Interest Rate (%)", fontsize=12)
ax2.grid(True, linestyle="--", alpha=0.3)

plt.tight_layout()
plt.show()


#### **📌 Key Insights: Interest Rate Distribution**

##### **Why did i pick this chart?**

The histogram shows how interest rates are distributed among borrowers, and the boxplot highlights the median, spread, and outliers. Together, they provide a clear view of typical interest rates and spotting any extreme values.

##### **What insights are found from the chart?**

* Most interest rates fall between 8% and 15%, showing the lender’s common pricing range.

* The distribution is slightly right-skewed, meaning a small number of loans carry very high interest rates.

* The boxplot confirms several high-interest outliers above 20%.

* The median interest rate is around 11–12%, which matches typical risk-based lending behavior.

##### **Will these insights help create a positive business impact?**

Yes. Understanding interest rate patterns helps the lender evaluate if pricing strategies align with borrower risk. It also helps in optimizing revenue, reducing risk exposure, and improving loan product structuring.

##### **Are there any insights indicating negative growth? Why?**

Yes. The presence of high-interest outliers could signal borrowers who are high-risk. Extremely high rates can also increase default probability because loans become harder to repay—leading to possible negative growth if not monitored.

## **DTI Rate Distribution**

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Histogram for DTI
sns.histplot(
    data=df,
    x='dti',
    bins=30,
    kde=True,
    linewidth=1.2,
    ax=ax1
)
ax1.set_title("Distribution of DTI", fontsize=14, weight="bold")
ax1.set_xlabel("DTI (%)", fontsize=12)
ax1.set_ylabel("Frequency", fontsize=12)
ax1.grid(True, linestyle="--", alpha=0.3)

# Boxplot for DTI
sns.boxplot(
    data=df,
    y='dti',
    linewidth=1.2,
    ax=ax2
)
ax2.set_title("Boxplot of DTI", fontsize=14, weight="bold")
ax2.set_ylabel("DTI (%)", fontsize=12)
ax2.grid(True, linestyle="--", alpha=0.3)

plt.tight_layout()
plt.show()


#### **📌 Key Insights: DTI (Debt-to-Income) Distribution**
##### **Why did I pick this chart?**

The histogram shows how borrowers’ DTI values are spread across the dataset, while the boxplot highlights the median, typical range, and possible high-risk borrowers.
Together, they help understand financial stress among borrowers and identify high-DTI segments prone to default.

##### **What insights are found from the chart?**

Most borrowers have a DTI between 8% and 18%, which represents the typical range for stable repayment ability.

The distribution is slightly right-skewed, indicating some borrowers carry significantly higher debt burdens.

The median DTI is around 13%, meaning half of the borrowers have manageable debt levels.

A noticeable number of borrowers have DTI values above 25%, signaling higher repayment risk.

The boxplot shows a few high-end values, but no extreme outliers beyond the whiskers.

##### **Will these insights help create a positive business impact?**

Yes.
Understanding DTI distribution helps lenders:

Identify high-risk borrowers before approval.

Optimize risk assessments for underwriting and pricing.

Support responsible lending by setting DTI thresholds to reduce future defaults.

This leads to healthier loan portfolios and fewer charge-offs.

##### **Are there any insights indicating negative growth? Why?**

Yes.
The presence of borrowers with very high DTI (above 25%) suggests financial stress and a higher chance of default.
If such loans are approved without proper checks, they may contribute to increasing default rates, hurting business performance.

## **Loan Status Distribution**

In [ ]:
plt.figure(figsize=(12, 6))

loan_status_counts = df['loan_status'].value_counts()
ax = sns.barplot(x=loan_status_counts.index, 
                 y=loan_status_counts.values,
                 palette='viridis')
plt.title('Distribution of Loan Status', pad=15, fontsize=12, fontweight='bold')
plt.xlabel('Loan Status', fontsize=10)
plt.ylabel('Count', fontsize=10)

for i, v in enumerate(loan_status_counts.values):
    ax.text(i, v, f'{v:,}', ha='center', va='bottom')

total = len(df)
for i, v in enumerate(loan_status_counts.values):
    percentage = (v/total)*100
    ax.text(i, v/2, f'{percentage:.1f}%', 
            ha='center', va='center',
            color='white', fontweight='bold')

plt.grid(axis='y', linestyle='--', alpha=0.3)
for spine in ['top', 'right']:
    ax.spines[spine].set_visible(False)
plt.tight_layout()
plt.show()

#### **📌 Key Insights:**

##### **Why did i pick this chart?**

A bar chart is the best way to compare the count and percentage of each loan status (Fully Paid, Charged Off, Current). It gives a clear view of portfolio performance and repayment behavior.

##### **What insights are found from the chart?**

* 83.3% loans are Fully Paid — majority of customers repay successfully.

* 13.8% loans are Charged Off — these represent completed defaults.

* 2.8% loans are Current — still in repayment and could turn into good or bad loans later.

* The large gap between Fully Paid and Charged Off shows good overall portfolio performance.

##### **Will these insights help create a positive business impact?**

Yes. Knowing how many loans default helps the lender strengthen underwriting rules, improve risk scoring, and take early action on high-risk customers. It also supports better forecasting and financial planning.

##### **Are there any insights indicating negative growth? Why?**

Yes. The 13.8% Charged Off segment indicates financial loss and higher credit risk. If this continues to increase, it can negatively impact profit margins. The lender must investigate which borrower or loan characteristics contribute to these defaults.

### **Numerical summary**

In [ ]:
print("\nLoan Status Distribution:")
print(loan_status_counts)
print("\nPercentage Distribution:")
print((loan_status_counts/total*100).round(2).astype(str) + '%')


## **2.BIVARIATE ANALYSIS**


### **Loan Status vs Loan Amount**

In [ ]:
plt.figure(figsize=(8,4))
sns.boxplot(
    x='loan_status',
    y='loan_amount',
    data=df,
    showfliers=False,      
    linewidth=1.2
)

plt.title("Loan Amount by Loan Status", fontsize=14, weight='bold')
plt.xlabel("Loan Status", fontsize=12)
plt.ylabel("Loan Amount", fontsize=12)

plt.grid(True, linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()

#### **📌 Key Insights: Loan Amount by Loan Status**

##### **Why did i pick this chart?**

A boxplot is useful here because it clearly compares the loan amount distributions across different loan statuses (Charged Off, Fully Paid, Current). It highlights differences in medians, ranges, and outliers in a simple visual format.

##### **What insights are found from the chart?**

* Current loans have the highest loan amounts, indicating borrowers with larger loans are still in repayment.

* Charged Off loans also show higher loan amounts compared to Fully Paid loans, suggesting larger loans may carry more default risk.

* Fully Paid loans have the lowest median loan amount, meaning smaller loans are more likely to be repaid successfully.

* There are significant outliers in all categories, especially for Current status.

##### **Will these insights help create a positive business impact?**

Yes. These insights help lenders identify how loan size affects repayment behavior. This can support better credit scoring, loan size limits, and risk-based underwriting policies.

##### **Are there any insights indicating negative growth? Why?**

Yes. The fact that Charged Off loans often have higher loan amounts suggests that high-value loans contribute disproportionately to losses. If not managed, this can harm profitability and increase overall portfolio risk.

### **Loan Status vs Interest Rate**

In [ ]:
plt.figure(figsize=(8,4))
sns.boxplot(
    x='loan_status',
    y='int_rate',
    data=df,
    showfliers=False,       
    linewidth=1.2
)

plt.title("Interest Rate by Loan Status", fontsize=14, weight='bold')
plt.xlabel("Loan Status", fontsize=12)
plt.ylabel("Interest Rate", fontsize=12)

plt.grid(True, linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()


#### **📌 Key Insights: Interest Rate by Loan Status**

##### **Why did i pick this chart?**

A boxplot is ideal for comparing interest rate distributions across different loan statuses. It shows the median, spread, and outliers, helping us identify how pricing differs between Fully Paid, Charged Off, and Current loans.

##### **What insights are found from the chart?**

* Charged Off loans have the highest median interest rates, indicating riskier borrowers are charged more—yet many still default.

* Fully Paid loans have the lowest interest rates, showing that borrowers with lower rates tend to repay successfully.

* Current loans also show high interest levels, suggesting these could become risky if borrowers struggle with repayment.

* There are several high-rate outliers across all categories, especially for Current loans.

##### **Will these insights help create a positive business impact?**

Yes. These insights help assess whether interest rate pricing aligns with actual repayment performance. It can support improving risk-based pricing, tightening high-risk approvals, and better portfolio monitoring.

##### **Are there any insights indicating negative growth? Why?**

Yes. The fact that many high-interest loans still end up Charged Off shows that raising interest rates alone does not reduce risk. High rates may increase financial pressure, making borrowers more likely to default and potentially harming profitability.

## **Loan Grade vs Loan Status**

In [ ]:
pd.crosstab(df['grade'], df['loan_status'], normalize='index').plot(
    kind='bar',
    figsize=(10,5),
    stacked=True
)
plt.title("Loan Status by Grade")
plt.ylabel("Percentage")
plt.xticks(rotation=0)
plt.show()


#### **📌 Key Insights: Loan Status by Grade**

##### **Why did you pick this chart?**

A stacked bar chart is ideal for comparing how loan outcomes (Fully Paid, Charged Off, Current) vary across borrower credit grades. It shows both the distribution within each grade and how risk increases from Grade A to Grade G.

##### **What insights are found from the chart?**

* Lower grades (A, B, C) have the highest proportion of Fully Paid loans, indicating low-risk borrowers.

* Higher grades (E, F, G) show a significant increase in Charged Off loans, confirming higher credit risk.

* Current loans also increase from A → G, meaning riskier borrowers are more likely to fall behind or struggle in repayment.

* Overall, default risk rises sharply as grade quality decreases.

##### **Will these insights help create a positive business impact?**

Yes. These insights validate the lender’s credit grading system. Identifying grades with higher default rates helps refine underwriting, adjust interest rates, and tighten approvals for higher-risk segments.

##### **Are there any insights indicating negative growth? Why?**

Yes. The high default percentage in grades F and G indicates potential financial losses. If many loans continue to be approved in these high-risk grades, the lender’s overall portfolio health and profitability may decline.

## **Purpose vs Loan Status**

In [ ]:
plt.figure(figsize=(17,8))
sns.countplot(
    data=df,
    x='purpose',
    hue='loan_status',
    linewidth=1.2
)

plt.title("Loan Purpose vs Loan Status", fontsize=16, weight='bold')
plt.xlabel("Loan Purpose", fontsize=12)
plt.ylabel("Count", fontsize=12)

plt.grid(True, linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()

#### **📌 Key Insights: Loan Purpose vs Loan Status**

##### **Why did you pick this chart?**

A grouped bar chart is ideal for comparing how loans of different purposes perform across the three statuses—Fully Paid, Charged Off, and Current. It highlights which loan purposes carry higher risk and which perform well.

##### **What insights are found from the chart?**

* Debt Consolidation dominates the dataset and also has the largest number of Charged Off loans.

* Credit Card, Other, Home Improvement, and Small Business loans also show noticeable defaults.

* Purposes like Car, Medical, Educational, and Wedding have relatively fewer defaults.

* Fully Paid loans remain the majority across most categories, but some purposes show higher risk than others.

##### **Will these insights help create a positive business impact?**

Yes. Understanding which loan purposes have higher default rates helps lenders adjust underwriting policies, interest rates, or eligibility criteria for those categories. It ensures better risk management and targeted interventions.

##### **Are there any insights indicating negative growth? Why?**

* Yes.Debt Consolidation shows both the highest volume and a high number of Charged Off loans, indicating potential risk concentration.

* Small Business and Other categories also show higher defaults relative to their size, which can harm profitability if not controlled.

These insights indicate the lender should monitor high-risk loan purposes closely to avoid long-term negative growth.

## **Loan Status vs DTI**

In [ ]:
plt.figure(figsize=(8,4))
sns.boxplot(
    x='loan_status',
    y='dti',
    data=df,
    showfliers=False,      
    linewidth=1.2
)

plt.title("DTI by Loan Status", fontsize=14, weight='bold')
plt.xlabel("Loan Status", fontsize=12)
plt.ylabel("DTI", fontsize=12)

plt.grid(True, linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()

#### **📌 Key Insights: DTI by Loan Status**

##### **Why did you pick this chart?**

A boxplot is ideal for comparing DTI distributions across different loan statuses. It visually highlights the median, spread, and variation in borrowers’ debt burden for Charged Off, Fully Paid, and Current loans.

##### **What insights are found from the chart?**

* Charged Off borrowers have higher median DTI, meaning they carry more debt relative to income.

* Fully Paid loans show the lowest median DTI, indicating stronger financial stability among successful payers.

* Current loans show a higher DTI spread, suggesting some of these borrowers may become future risks.

* A DTI above ~0.15 is common in Charged Off loans, showing a clear risk pattern.

##### **Will these insights help create a positive business impact?**

Yes. This insight helps the lender refine credit policies by setting appropriate DTI thresholds. It improves risk assessment and reduces the likelihood of approving loans for financially stressed borrowers.

##### **Are there any insights indicating negative growth? Why?**

Yes. The higher DTI levels in Charged Off and Current loans indicate that debt-heavy borrowers are more likely to default. If such customers continue to be approved, overall portfolio risk and losses may increase.

### **Home Ownership vs Loan Amount**

In [ ]:
plt.figure(figsize=(8,4))
sns.boxplot(
    x='home_ownership',
    y='loan_amount',
    data=df,
    showfliers=False,     
    linewidth=1.2
)

plt.title("Loan Amount by Home Ownership Type", fontsize=14, weight='bold')
plt.xlabel("Home Ownership", fontsize=12)
plt.ylabel("Loan Amount", fontsize=12)

plt.grid(True, linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()

#### **📌 Key Insights: DTI by Loan Status**

##### **Why did you pick this chart?**

A boxplot is ideal for comparing loan amount distributions across different home ownership categories. It visualizes the median, spread, and typical loan size for each group, helping identify which borrower types request larger or smaller loans.

##### **What insights are found from the chart?**

* Mortgage holders take the highest loan amounts, indicating stronger credit confidence or higher financial needs.

* Borrowers who Rent or Own homes take moderate loan amounts, with similar median values.

* People with ‘None’ as home ownership take the smallest loans, likely due to lower financial stability or limited collateral.

* The range is widest for Mortgage, showing both small and very large loan requests.

##### **Will these insights help create a positive business impact?**

Yes. Understanding how loan amounts vary by home ownership helps lenders tailor loan products, adjust approval criteria, and better assess risk for different customer profiles.

##### **Are there any insights indicating negative growth? Why?**

Yes. Borrowers with no home ownership (None) typically request smaller loans, which may indicate lower creditworthiness. Additionally, the large variability among Mortgage borrowers could introduce risk for high-value loans if not evaluated properly.

### **Heatmap**

In [ ]:
numerical_cols = [
    "annual_income",
    "dti",
    "installment",
    "int_rate",
    "loan_amount",
    "total_acc",
    "total_payment"
]

corr_matrix = df[numerical_cols].corr()

plt.figure(figsize=(10,6))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", linewidths=0.5)
plt.title("Correlation Heatmap of Numerical Features", fontsize=14)
plt.show()

#### **📌 Key Insights:Correlation Heatmap**
##### **Why did i pick this chart?**

A correlation heatmap visually shows the strength and direction of relationships between numerical variables. It helps quickly identify which features are strongly related and which ones may influence loan performance or modeling.

##### **What insights are found from the chart?**
##### **Strong Positive Correlations:**

* Loan Amount ↔ Installment (0.93) -> Bigger loans naturally have higher monthly installments.

* Loan Amount ↔ Total Payment (0.89) -> Higher loan amounts result in higher total repayment.

* Installment ↔ Total Payment (0.86) -> Monthly payment and total repayment move together.

##### **Moderate Positive Correlations:**

* Interest Rate ↔ Loan Amount (0.31)-> Higher loan amounts tend to have slightly higher interest rates.

* Total Accounts ↔ Loan Amount (~0.26)-> More financially active customers may take larger loans.

##### **Weak or No Correlations:**

* DTI has very weak correlations with most variables.

* Interest Rate and Total Accounts (-0.04) show almost no relationship.

* Income shows weak positive correlations, meaning income alone does not heavily drive loan size or interest rate.

##### **Will these insights help create a positive business impact?**

Yes. Understanding correlations helps:

* Improve predictive modeling (selecting impactful features).

* Reduce multicollinearity for machine learning models.

* Understand borrower behavior—for example, high loan amounts → higher installments → higher total payments.

##### **Are there any insights indicating negative growth? Why?**

Yes.

* The weak correlation between income and DTI/loan amount suggests some borrowers may be taking loans beyond their financial capacity.

* Higher interest rates only weakly correlate with borrower characteristics, implying rates may not fully reflect risk, potentially contributing to defaults.

* These insights point to potential mismatches in pricing and underwriting that can negatively impact portfolio performance.


##  **6.Feature_engineering**


### **Date Feature Extraction**

In [ ]:
df['issue_year'] = df['issue_date'].dt.year
df['issue_month'] = df['issue_date'].dt.month

### **Income Bucketing**

In [ ]:
df['income_bucket'] = pd.cut(df['annual_income'],
                              bins=[0,50000,100000,200000,500000],
                              labels=['Low','Medium','High','Very High'])

### **Income-to-Loan Ratio**

In [ ]:
df['income_to_loan_ratio'] = df['annual_income'] / df['loan_amount']

### **DTI Bucketing**

In [ ]:
df['dti_bucket'] = pd.cut(
    df['dti'].fillna(-1),
    bins=[0, 10, 20, 30, 100],
    labels=['Low', 'Medium', 'High', 'Very High']
)

### **Interest Rate Bucketing**

In [ ]:
df['int_rate_bucket'] = pd.cut(
    df['int_rate'].fillna(-1),
    bins=[0, 10, 15, 20, 100],
    labels=['Low', 'Medium', 'High', 'Very High']
)


## **7.Segmentation Analysis**


In [ ]:
# ------------------------------------------------
# Common default labels: 'Charged Off', 'Default'
# ------------------------------------------------

df['is_default'] = df['loan_status'].isin(['Charged Off', 'Default'])


### **A. Borrower Risk Segmentation**



#### **1.DTI Segmentation (Debt-to-Income)**

In [ ]:
df['dti_group'] = pd.cut(
    df['dti'].fillna(-1),
    bins=[0, 0.10, 0.20, 0.30, 1],
    labels=['0-10%', '10-20%', '20-30%', '>30%']
  
)


dti_seg = df.groupby('dti_group').agg(
    loan_count=('loan_status', 'count'),
    default_count=('loan_status', lambda x: (x.isin(['Charged Off', 'Default', 'Late'])).sum()),
    avg_int_rate=('int_rate', 'mean'),
    avg_dti=('dti', 'mean'),
    avg_loan_amount=('loan_amount', 'mean')
)

# 3. Default rate %
dti_seg['default_rate_pct'] = (
    dti_seg['default_count'] / dti_seg['loan_count'] * 100
)

# 4. Sort by risk (highest default rate first) + round
dti_seg = dti_seg.sort_values('default_rate_pct', ascending=False)
dti_seg = dti_seg.round(2)

dti_seg

In [ ]:
colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(dti_seg)))

plt.figure(figsize=(12, 5))
plt.bar(dti_seg.index.astype(str), dti_seg['default_rate_pct'], color=colors)

plt.title('Default Rate by DTI Group', fontweight='bold')
plt.xlabel('DTI Group', fontweight='bold')
plt.ylabel('Default Rate (%)', fontweight='bold')

for i, v in enumerate(dti_seg['default_rate_pct']):
    plt.text(i, v, f'{v:.1f}%', ha='center', va='bottom', fontweight='bold')

plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

#### **📌 Key Insights: Default Rate by DTI (Debt-to-Income) Range**

##### **Why did you pick this chart?**

This bar chart is ideal for comparing default risk across different DTI ranges. It shows how increasing debt burden affects the likelihood of default.

##### **what insights are found from the chart?**

* Higher DTI = Higher Default Rate

* The 20–30% DTI group has the highest default rate (15.6%).

* The 10–20% group also shows elevated default risk (14.4%).

* Borrowers in the 0–10% DTI group have the lowest default rate (11.9%).

* The trend is clear: as DTI increases, default risk rises.

##### **Will these insights help create a positive business impact?**

Yes. These insights help lenders:

* Set DTI thresholds during loan approval

* Identify high-risk borrower segments

* Adjust interest rates based on borrower financial stress levels

* This leads to lower losses and better loan portfolio performance.

##### **Are there insights indicating negative growth?**

Yes.

Higher default rates in the 20–30% DTI segment indicate:

* Borrowers with high debt relative to income are struggling

* Approving loans in this range can increase financial losses

* Without strict policies, portfolio risk may grow over time

### **2.Income segmentation**

In [ ]:
if 'annual_income' in df.columns:
    df['income_group'] = pd.cut(
        df['annual_income'].fillna(0),
        bins=[-1, 50_000, 100_000, 150_000, 1_000_000],
        labels=['<=50k', '50k-100k', '100k-150k', '>150k']
)

# Segmentation table (same pattern as grade_seg)
income_seg = df.groupby('income_group').agg(
    loan_count=('loan_status', 'count'),
    default_count=('loan_status', lambda x: (x.isin(['Charged Off', 'Default', 'Late'])).sum()),
    avg_int_rate=('int_rate', 'mean'),
    avg_dti=('dti', 'mean'),
    avg_loan_amount=('loan_amount', 'mean')
)

# Default rate %
income_seg['default_rate_pct'] = (
    income_seg['default_count'] / income_seg['loan_count'] * 100
)

# Sort by risk + round
income_seg = income_seg.sort_values('default_rate_pct', ascending=False)
income_seg = income_seg.round(2)

income_seg

In [ ]:
# --- Default Rate by Income Group Visual ---

income_plot = income_seg.sort_values("default_rate_pct", ascending=False)

plt.figure(figsize=(12,5))
sns.barplot(
    data=income_plot,
    x=income_plot.index,
    y='default_rate_pct',
    hue=income_plot.index,     # needed to use palette
    palette="viridis",
    dodge=False,
    legend=False
)

plt.title("Default Rate by Income Segment", fontsize=14, weight='bold')
plt.xlabel("Income Group", fontsize=12)
plt.ylabel("Default Rate (%)", fontsize=12)

# Add bold labels on bars
for i, v in enumerate(income_plot['default_rate_pct']):
    plt.text(
        i, v, f"{v:.1f}%",
        ha='center',
        va='bottom',
        fontsize=10,
        fontweight='bold'
    )

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


#### **📌 Key Insight:Default Rate by Income Category**

##### **Why did you pick this chart?**

A bar chart is perfect for comparing how default rates change across different income groups. It clearly shows whether higher or lower income levels impact the likelihood of loan repayment.

##### **What insights are found from the chart?**

* Borrowers earning ≤50k have the highest default rate (16%) — indicating strong financial stress in this group.

* Default rate drops steadily as income increases.

* Borrowers earning 50k–100k default at 13.1%, showing moderate risk.

* Borrowers earning above 100k have the lowest default rate (10.4%).

* Overall, higher income → lower default risk.

##### **Will these insights help create a positive business impact?**

Yes. Income-based segmentation allows lenders to:

* Improve risk scoring models

* Set minimum income thresholds

* Adjust loan amounts and interest rates based on borrower affordability

* Reduce exposure to high-risk income groups

* This leads to more stable and profitable lending decisions.

##### **Are there insights indicating negative growth? Why?**

Yes. The ≤50k income segment has a significantly higher default rate.
If the lender continues issuing large volumes of loans to this group without proper checks, it can result in:

* Higher losses

* Increased portfolio risk

* Lower long-term profitability

* This segment requires stricter underwriting or smaller loan limits.

## **B. Loan Characteristics Segmentation**

### **1. INTEREST RATE SEGMENTATION**

In [ ]:
df['int_rate_group'] = pd.cut(
    df['int_rate'].fillna(-1),
    bins=[0, 0.10, 0.15, 0.20, 1],
    labels=['<10%', '10-15%', '15-20%', '>20%']
)
int_seg = df.groupby('int_rate_group').agg(
    loan_count=('loan_status', 'count'),
    default_count=('loan_status', lambda x: (x.isin(['Charged Off','Default','Late'])).sum()),
    avg_loan_amount=('loan_amount', 'mean'),
    avg_dti=('dti', 'mean')
)

int_seg['default_rate_pct'] = (int_seg['default_count'] / int_seg['loan_count']) * 100
int_seg = int_seg.round(2)
int_seg


In [ ]:
int_plot = int_seg.sort_values("default_rate_pct", ascending=False)

plt.figure(figsize=(12, 5))
sns.set_style("whitegrid")

# Bar plot (viridis palette)
sns.barplot(
    x=int_plot.index.astype(str),
    y=int_plot["default_rate_pct"],
    palette="viridis"
)

# Titles and labels
plt.title("Default Rate by Interest Rate Bucket", fontsize=14, fontweight='bold')
plt.xlabel("Interest Rate Bucket", fontsize=12, fontweight='bold')
plt.ylabel("Default Rate (%)", fontsize=12, fontweight='bold')

# Bold % labels above bars
for i, v in enumerate(int_plot["default_rate_pct"]):
    plt.text(
        i, v,
        f"{v:.1f}%",
        ha='center', va='bottom',
        fontsize=11, fontweight='bold'
    )

# Grid
plt.grid(axis='y', linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

##### **1. Why did i pick this specific chart?**

I used a bar chart because it clearly compares default rates across discrete interest-rate buckets.
It helps show how risk changes as interest rates increase, which is essential for credit-risk segmentation.

##### **2. What insights did you find?**

* Borrowers with >20% interest rate have the highest default rate (~33%) — very high-risk segment.

* The 15–20% group also shows elevated defaults (~23%).

* Borrowers in the 10–15% bucket have a moderate default rate (~14%).

* Borrowers with <10% interest rate have the lowest default rate (~6%).

Conclusion:
Higher interest rates = higher probability of default.
This shows that risk-based pricing by lenders is aligned with actual borrower behavior.

##### **3. Will these insights create a positive business impact?**

Yes.
This segmentation helps lenders:

* Optimize loan pricing for different risk tiers.

* Identify high-risk customer groups requiring stricter approval criteria.

* Improve loss forecasting and reserve planning.

* Design targeted risk-reduction strategies (e.g., income verification, collateral).

* This leads to lower default losses and better portfolio quality.

##### **4. Any insights that indicate negative growth? Justify.**

Yes — the high-interest buckets (>15%) show very high default rates.

Why this is negative:

* These segments contribute disproportionately to loan losses.

* High interest rates may be pushing financially vulnerable borrowers into default.

* If the lender expands aggressively in these segments, it can increase NPA levels and reduce profitability.

Thus, the bank should control exposure in the higher-interest-rate buckets.

### **2. Loan Amount Segmentation**

In [ ]:
df['loan_amt_bin'] = pd.cut(
    df['loan_amount'],
    bins=[0,5000,10000,15000,20000,100000],
    labels=['0-5k','5k-10k','10k-15k','15k-20k','20k+']
)

loan_amt_seg = df.groupby('loan_amt_bin').agg(
    loan_count=('loan_status','count'),
    default_count=('loan_status', lambda x: (x.isin(['Charged Off','Default','Late'])).sum()),
    avg_int_rate=('int_rate','mean'),
    avg_dti=('dti','mean')
)

loan_amt_seg['default_rate_pct'] = (loan_amt_seg['default_count']/loan_amt_seg['loan_count'])*100
loan_amt_seg = loan_amt_seg.round(2)
loan_amt_seg


In [ ]:
# --- Default Rate by Loan Amount Bucket Visual ---

loan_plot = loan_amt_seg.sort_values("default_rate_pct", ascending=True)

plt.figure(figsize=(12,5))
sns.set_style("whitegrid")

sns.barplot(
    data=loan_plot,
    x=loan_plot.index,
    y="default_rate_pct",
    hue=loan_plot.index,       # enables palette
    palette="viridis",
    dodge=False,
    legend=False
)

plt.title("Default Rate by Loan Amount Bucket", fontsize=14, weight='bold')
plt.xlabel("Loan Amount Bucket", fontsize=12)
plt.ylabel("Default Rate (%)", fontsize=12)

# Add bold labels on bars
for i, v in enumerate(loan_plot['default_rate_pct']):
    plt.text(
        i, v, f"{v:.1f}%",
        ha='center',
        va='bottom',
        fontsize=10,
        fontweight='bold'
    )

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


#### **📌 Key Insight:**
##### **1. Why did i pick this specific chart?**

A bar chart is ideal for comparing default rates across different loan amount buckets.
It visually shows how risk changes as loan size increases, making it easy to detect patterns.

##### **2. What insights are found from the chart?**

* Borrowers with larger loan amounts show higher default rates.

* Default rates rise from 12–13% for smaller loans (0–15k)
to 16%–18.5% for larger loans (15k–20k and 20k+).

* The highest-risk group is 20k+, with an 18.5% default rate.

Conclusion:
* As loan amount increases, borrower repayment risk increases.

##### **3. Will these insights help create a positive business impact?**

Yes.
This analysis helps lenders:

* Adjust credit policies for large loan requests.

* Introduce stricter verification for high-amount borrowers.

* Improve risk-based pricing and limit setting.

* Reduce losses by controlling exposure to larger, riskier loans.

* This directly improves portfolio stability and profitability.

##### **4. Any insights that lead to negative growth? Justify.**

Yes — the loan amount buckets 15k–20k and especially 20k+ show significantly higher default rates.

Why this is negative:

* Large loans have higher loss severity if they default.

* High default rates in these buckets can increase NPAs, hurting portfolio performance.

* If the organization aggressively grows this segment without proper controls,it may face rising credit losses and reduced margins.

### **3.Term (36 vs 60 months)**

In [ ]:
term_seg = df.groupby('term').agg(
    loan_count=('loan_status','count'),
    default_count=('loan_status', lambda x: (x.isin(['Charged Off','Default','Late'])).sum()),
    avg_int_rate=('int_rate','mean'),
    avg_dti=('dti','mean'),
    avg_loan_amount=('loan_amount','mean')
)

# Default Rate
term_seg['default_rate_percentage'] = (term_seg['default_count'] / term_seg['loan_count']) * 100

# Round values
term_seg = term_seg.round(2)

term_seg

In [ ]:
term_plot = term_seg.sort_values("default_rate_percentage", ascending=False)

plt.figure(figsize=(10,5))
sns.set_style("whitegrid")

sns.barplot(
    data=term_plot,
    x=term_plot.index,
    y='default_rate_percentage',
    hue=term_plot.index,
    palette="viridis",
    dodge=False,
    legend=False
)

plt.title("Default Rate by Loan Term", fontsize=14, weight='bold')
plt.xlabel("Loan Term", fontsize=12)
plt.ylabel("Default Rate (%)", fontsize=12)

# Add bold labels on bars (your requested format)
for i, v in enumerate(term_plot['default_rate_percentage']):
    plt.text(
        i, v, f"{v:.1f}%",
        ha='center',
        va='bottom',
        fontsize=10,
        fontweight='bold'
    )

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

#### **📌 Key Insights: Default % by Loan Term**

##### **1. Why did i pick this specific chart?**

A bar chart is the simplest way to compare default rates between the two loan terms (36 vs 60 months).
It clearly highlights the difference in risk across loan durations.

##### **2. What insights are found from the chart?**

* 60-month loans have a much higher default rate (22.3%) than
* 36-month loans (10.7%) — almost double.

Longer-term borrowers are riskier due to longer repayment pressure and higher interest accumulation.

##### **3. Will the insights help create a positive business impact?**

Yes.
Lenders can:

* Tighten approval criteria for 60-month loans.

* Apply higher risk-based interest rates for long-term borrowers.

* Promote 36-month loans to improve repayment performance.

* This reduces credit losses and strengthens the loan portfolio.

##### **4. Any insights that lead to negative growth? Justify.**

Yes — the 60-month loan term shows a very high default rate.

* Reason for negative impact

* Longer terms increase borrower fatigue and repayment uncertainty.

* Missed EMIs accumulate, increasing the chance of charge-offs.

* If the business pushes more 60-month loans, overall NPA levels will rise.

### **4.GRADE SEGMENTATION**

In [ ]:
grade_seg = df.groupby('grade').agg(
    loan_count=('loan_status','count'),
    default_count=('loan_status', lambda x: (x.isin(['Charged Off','Default','Late'])).sum()),
    avg_int_rate=('int_rate','mean'),
    avg_dti=('dti','mean'),
    avg_loan_amount=('loan_amount','mean')
)

grade_seg['default_rate_pct'] = (grade_seg['default_count']/grade_seg['loan_count'])*100

# Sort by risk
grade_seg = grade_seg.sort_values('default_rate_pct', ascending=False)
grade_seg = grade_seg.round(2)
grade_seg

In [ ]:
# --- Grade/Subgrade Segmentation Visual ---
grade_plot = grade_seg.sort_values("default_rate_pct", ascending=False)

plt.figure(figsize=(12,5))
sns.barplot(
    data=grade_plot,
    x=grade_plot.index,
    y='default_rate_pct',
    palette="viridis"
)

plt.title("Default Rate by grade",fontsize=14, weight='bold')
plt.xlabel("Grade",fontsize=12)
plt.ylabel("Default Rate (%)",fontsize=12)

for i, v in enumerate(grade_plot['default_rate_pct']):
    plt.text(i, v, f"{v:.1f}%", ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


#### **📌 Key Insights:**
##### **1. Why did i pick this specific chart?**

A bar chart is ideal to compare default rates across ordered categories (A → G).
It clearly highlights the risk pattern across different credit subgrades.

##### **2. What insights are found from the chart?**

* Default rate increases as credit subgrade worsens.

* A-grade borrowers have the lowest default rate (5.7%).

* G-grade borrowers have the highest default rate (31.3%).

* There is a consistent upward trend from A → G, confirming that credit subgrades accurately reflect borrower risk.

##### **3. Will the insights help create a positive business impact?**

Yes.
These insights help lenders:

* Strengthen risk-based pricing (higher rates for lower grades).

* Adjust approval thresholds for F/G borrowers.

* Build better credit scoring and underwriting models.

* Reduce portfolio-level credit losses by controlling high-risk segments.

##### **4. Any insights that lead to negative growth? Justify.**

Yes — Subgrades E, F, and G show very high default rates (25%–31%).

Reason for negative impact

* Lending more to F/G-grade borrowers increases charge-offs, NPAs, and operational risk.

* These categories require stricter approval criteria, higher collateral requirements, or smaller loan amounts to minimize losses.

### **C. Behavioral Segmentation**

### **1. Loan Purpose Segmentation**

In [ ]:
purpose_seg = df.groupby('purpose').agg(
    loan_count=('loan_status','count'),
    default_count=('loan_status', lambda x: (x.isin(['Charged Off','Default','Late'])).sum()),
    avg_int_rate=('int_rate','mean'),
    avg_dti=('dti','mean'),
    avg_loan_amount=('loan_amount','mean')
)

purpose_seg['default_rate_percentage'] = (purpose_seg['default_count']/purpose_seg['loan_count'])*100



# Sort by risk
purpose_seg = purpose_seg.sort_values('default_rate_percentage', ascending=False)
purpose_seg = purpose_seg.round(2)
purpose_seg


In [ ]:
# --- Default Rate by Loan Purpose Visual ---

purpose_plot = purpose_seg.sort_values("default_rate_percentage", ascending=False)

plt.figure(figsize=(16,6))
sns.barplot(
    data=purpose_plot,
    x=purpose_plot.index,
    y='default_rate_percentage',
    hue=purpose_plot.index,       # needed for palette
    palette="viridis",
    dodge=False,
    legend=False
)

plt.title("Default Rate by Loan Purpose", fontsize=14, weight='bold')
plt.xlabel("Loan Purpose", fontsize=12)
plt.ylabel("Default Rate (%)", fontsize=12)

# Add bold labels on bars
for i, v in enumerate(purpose_plot['default_rate_percentage']):
    plt.text(
        i, v, f"{v:.1f}%",
        ha='center',
        va='bottom',
        fontsize=10,
        fontweight='bold'
    )

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


#### **📌 Key Insights: Default % by Purpose** 

##### **1. Why did i pick this chart?**

A bar chart works best because:

* It allows easy comparison of default rates across multiple loan purposes.

* The categories are discrete (business, education, medical, car, etc.).

* It visually highlights high-risk vs low-risk purposes instantly.

##### **2. What insights are found from the chart?**
🔍 High-Risk Loan Purposes

* Small business loans → 25.6% default rate (highest).

* Renewable energy, educational, house loans → 15–18% default rate.


🔍 Moderate-Risk Loan Purposes

* Medical, moving, other, debt consolidation → 14–15% default rates.

🔍 Low-Risk Loan Purposes

* Home improvement (11.4%)

* Car loans (10.3%)

* Credit card refinancing (10.2%)

* Major purchase (9.8%)

* Wedding loans (9.3%) — lowest default rate

* Borrowers taking car, credit card, and wedding loans are less likely to default.

##### **3. Will the insights help create a positive business impact?**

Yes.
These insights help lenders:

Risk-Based Pricing

* Charge higher interest rates or require stricter documentation for small business, renewable energy, and education loans.

Portfolio Optimization

* Increase exposure to low-risk categories (wedding, car, credit card refinancing).

These actions reduce losses and improve portfolio health.

##### **4. Any insights that lead to negative growth? Justify.**
Yes — Small business loans lead to negative growth.

Why?

* They have the highest default rate (25.6%).

* Small businesses often face cash flow uncertainty.

* Economic downturns disproportionately affect small businesses.

* High exposure to this category increases total charge-offs, collection costs, and NPA ratios.

Other potential negative-impact categories:

* Renewable energy (18.1%) – new industry, unstable returns

* Educational loans (15.9%) – no immediate income generation

* House improvement loans (15.6%) – often taken by cash-stressed individuals

These need stricter lending guidelines to prevent future losses.

### **2. State-wise segmentation**

In [ ]:
# State-wise segmentation
state_seg = df.groupby('address_state').agg(
    loan_count=('loan_status', 'count'),
    default_count=('loan_status', lambda x: (x.isin(['Charged Off', 'Default', 'Late'])).sum()),
    avg_int_rate=('int_rate', 'mean'),
    avg_dti=('dti', 'mean'),
    avg_loan_amount=('loan_amount', 'mean')
)

# Default rate %
state_seg['default_rate_percentage'] = (state_seg['default_count'] / state_seg['loan_count']) * 100

# Round numeric values
state_seg = state_seg.round(2)

# Sort by risk (highest default rate first)
state_seg = state_seg.sort_values('default_rate_percentage', ascending=False)

state_seg


In [ ]:
state_plot = state_seg.head(10)

plt.figure(figsize=(12,6))
sns.set_style("whitegrid")

sns.barplot(
    data=state_plot,
    x=state_plot.index,
    y="default_rate_percentage",
    hue=state_plot.index,
    palette="viridis",
    dodge=False,
    legend=False
)

plt.title("Top 10 States by Default Rate", fontsize=14, weight='bold')
plt.xlabel("State", fontsize=12)
plt.ylabel("Default Rate (%)", fontsize=12)

# Add bold labels on bars
for i, v in enumerate(state_plot['default_rate_percentage']):
    plt.text(
        i, v, f"{v:.1f}%",
        ha='center',
        va='bottom',
        fontsize=10,
        fontweight='bold'
    )

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


#### **📌 Key Insight**

##### **1. Why did you pick this chart?**

* A bar chart is ideal for comparing default rates across multiple states.

* It clearly highlights which states contribute disproportionately to credit risk.

* Easy to identify top high-risk regions at a glance — important for lending decision-making.

##### **2. What insights are found from the chart?** 

* NE shows an extremely high default rate (60%), much higher than all other states — a clear outlier.

* NV (20.9%), AK (19.2%), SD (17.5%) also show significantly high risk.

* The remaining states in the top 10 fall between 15–17%, indicating moderately high risk.

* Overall, the chart reveals geographical clustering of defaults.

##### **3. Will these insights help create a positive business impact?** 

Yes. These insights help lenders:

* Apply location-based risk scoring.

* Adjust interest rates or credit limits by state.

* Strengthen collection strategy in high-risk states.

* Reduce future losses by avoiding aggressive lending in risk-heavy regions.

This leads to better risk management, lower NPA levels, and improved portfolio quality.

##### **4. Are there insights that indicate negative growth? Justify.** 

Yes — the extremely high default rate in NE (60%).

* This suggests severe borrower instability, weak economic conditions, or historical loan mismanagement in that area.

* Lending heavily in such a state may increase default risk, causing negative financial impact for the business.

* Similar concerns apply (to a lesser extent) to NV, AK, and SD.


##  **8.Cohort Analysis**


### **Create Issue Month & Monthly Metrics** 

In [ ]:
# 1. Create issue_month (YYYY-MM format) 
if 'issue_date' in df.columns:
    df['issue_month'] = df['issue_date'].dt.to_period('M').astype(str) 

# 2. Simple cohort table: loans per issue_month and default rate
if 'issue_month' in df.columns:
    cohort_basic = df.groupby('issue_month').agg(
        loan_count = ('id','count'),
        avg_loan_amount = ('loan_amount','mean'),
        default_rate = ('is_default','mean')
    ).sort_index()  # sorts by month ascending
    cohort_basic['default_rate_pct'] = cohort_basic['default_rate'] * 100
    display(cohort_basic.round(3))


In [ ]:
# Default Rate by Issue Month (Cohort)
if 'issue_month' in df.columns:
    plt.figure(figsize=(10,4))
    plt.plot(
        cohort_basic['default_rate_pct'],
        marker='o',
        linewidth=2,
        markersize=6
    )

    plt.title("Default Rate by Issue Month", fontsize=14, weight='bold')
    plt.xlabel("Issue Month", fontsize=12, weight='bold')
    plt.ylabel("Default Rate (%)", fontsize=12, weight='bold')

    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()


#### **📌 Key Insights**

##### **Why did you pick this chart?**

I chose a line chart because:

* It is the best visual to show trend over time.

* Helps identify seasonality, month-to-month fluctuations, or emerging risk patterns.

* Cohort analysis becomes meaningful only when visualized chronologically.


#### **Insights from the chart**

* Default rates fluctuate monthly between ~11.5% and ~15.1%.

Lowest default rate:
* February 2021 – ~11.6%

Highest default rate:
* May 2021 and December 2021 – ~15.1%

* A sharp increase happens from April → May (12.8% → 15.1%).

* Late-year months (Sep–Dec) show consistently higher defaults.

* This indicates possible seasonal or economic cycle impact (holidays, expenses, liquidity issues).

Overall Trend

* There is a slight upward movement in default rates across the year.

* Borrower risk gradually increased over time.

##### **Will these insights help create a positive business impact?**
Yes — very strongly.

This analysis enables the business to:

Improve Risk Forecasting

* Month-level trends allow lenders to predict upcoming risky periods.

Dynamic Interest Rate / Limit Adjustment

* High-risk months → apply precautionary measures:

* Higher interest rates for subprime borrowers

* Stricter credit checks

* Lower loan amounts

Better Portfolio Planning

* Helps the business prepare for expected spikes in delinquencies, improving cashflow planning.


##### **Any insights that indicate negative growth?**
Yes — Some months signal negative patterns

The increase in defaults during May, September, October, December points to:

* Borrower stress

* Seasonal expenses (festivals, year-end bills)

* Economic fluctuations



### **Overall loan default rate, and how has it changed by issue_year and issue_month**

In [ ]:
default_rate = (df['loan_category'] == 'Bad Loan').mean()
print(f"\nDefault Rate: {default_rate:.2%}")

default_by_year = df.groupby('issue_year')['is_default'].mean() * 100
print(default_by_year)


# Default Rate by Month
df['issue_month'] = df['issue_date'].dt.to_period('M')

default_by_month = df.groupby('issue_month')['is_default'].mean() * 100
default_by_month = default_by_month.round(2)
default_by_month

## **9.Recovery Insights**

In [ ]:
# Filter only defaulted / charged-off loans
df_defaulted = df[df['loan_status'].isin(['Charged Off', 'Default'])].copy()

# Calculate recovery rate percentage
df_defaulted['recovery_rate_pct'] = (df_defaulted['total_payment'] / df_defaulted['loan_amount']) * 100

# Round for better readability
df_defaulted['recovery_rate_pct'] = df_defaulted['recovery_rate_pct'].round(2)

df_defaulted[['loan_amount', 'total_payment', 'recovery_rate_pct']].head()


In [ ]:
overall_recovery_rate = (
    df_defaulted['total_payment'].sum() /
    df_defaulted['loan_amount'].sum()
) * 100

overall_recovery_rate = overall_recovery_rate.round(2)
overall_recovery_rate


### **Recovery Rate by Loan Purpose**

In [ ]:
recovery_purpose = df_defaulted.groupby('purpose').agg(
    defaulted_loans=('id', 'count'),
    total_recovered=('total_payment', 'sum'),
    total_defaulted_amount=('loan_amount', 'sum')
)

# Convert to millions
recovery_purpose['total_recovered_mn'] = recovery_purpose['total_recovered'] / 1_000_000
recovery_purpose['total_defaulted_amount_mn'] = recovery_purpose['total_defaulted_amount'] / 1_000_000

# Calculate recovery rate %
recovery_purpose['recovery_rate_pct'] = (
    recovery_purpose['total_recovered'] /
    recovery_purpose['total_defaulted_amount']
) * 100

# Keep final clean columns
recovery_purpose = recovery_purpose[[
    'defaulted_loans',
    'total_recovered_mn',
    'total_defaulted_amount_mn',
    'recovery_rate_pct'
]]

recovery_purpose = recovery_purpose.round(2)
recovery_purpose


### **Recovery Rate by Grade**

In [ ]:
recovery_grade = df_defaulted.groupby('grade').agg(
    recovered=('total_payment', 'sum'),
    default_amount=('loan_amount', 'sum')
)

# Convert amounts into millions
recovery_grade['recovered_mn'] = recovery_grade['recovered'] / 1_000_000
recovery_grade['default_amount_mn'] = recovery_grade['default_amount'] / 1_000_000

# Recovery rate
recovery_grade['recovery_rate_pct'] = (
    recovery_grade['recovered'] / recovery_grade['default_amount']
) * 100

# Keep clean final columns
recovery_grade = recovery_grade[[
    'recovered_mn',
    'default_amount_mn',
    'recovery_rate_pct'
]]

recovery_grade = recovery_grade.round(2)
recovery_grade


## **10.Solution to Business Objective**

The objective is to analyze the loan portfolio and identify factors driving defaults, borrower risk patterns, and overall portfolio performance.

We cleaned the data, validated financial fields, performed univariate/bivariate EDA, studied customer characteristics, analyzed default trends, built cohort insights, and identified key risk drivers such as high DTI, low income, risky sub-grades, and high loan amounts.

This analysis provides a clear view of borrower quality, repayment behavior, and portfolio risks.

### **What Do You Suggest to the Client to Achieve Their Business Objective?**

### **Final Recommendations**

Tighten loan approval rules for high-risk segments (low grades: E, F, G).

Increase interest rates or add stricter checks for borrowers with high DTI or high loan amounts.

Strengthen verification for high-risk loan purposes (Debt Consolidation, Small Business, Other).

Promote shorter loan terms (36-month) to reduce default probability.

Use automated alerts for risky profiles (high DTI, high interest rate, long loan term).



# **✅ Conclusion**

##### ***The loan portfolio analysis clearly highlights the key factors influencing borrower risk and default behavior. High DTI, low income, lower credit grades, and larger loan amounts consistently show higher default tendencies. Trend and cohort analysis further reveal patterns in issue months and repayment behavior, enabling better forecasting of future risk.***

##### ***Overall, this study provides a data-driven understanding of borrower quality and portfolio health. By tightening improving risk-based pricing, strengthening early monitoring, and adopting predictive models such as PD the client can significantly reduce default rates and improve overall loan performance. The insights generated form a strong foundation for smarter lending decisions, proactive risk management, and long-term portfolio stability***